<a href="https://colab.research.google.com/github/LoQiseaking69/TimeRing-/blob/main/rm_ringltheory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, Output, VBox, FloatSlider
import pandas as pd
from scipy.stats import skew, kurtosis

class InteractiveRingLaserViewer:
    def __init__(self, wavelength=1550e-9, beam_power=1e-3, ring_radius=1e9, num_detectors=360, c=299792458, G=6.67430e-11):
        self.wavelength = wavelength
        self.beam_power = beam_power
        self.ring_radius = ring_radius
        self.num_detectors = num_detectors
        self.c = c
        self.G = G

        self.output = Output()
        self.laser_energy_slider = FloatSlider(min=0, max=1e6, step=1, value=1e5, description='Laser Energy:')
        self.laser_radius_slider = FloatSlider(min=0, max=1e6, step=1, value=1e5, description='Laser Radius:')
        self.angular_momentum_slider = FloatSlider(min=0, max=1e6, step=1, value=1e5, description='Angular Momentum:')
        self.pulse_duration_slider = FloatSlider(min=0, max=1e6, step=1, value=1e5, description='Pulse Duration:')

        self.setup_interactive()

    def setup_interactive(self):
        @interact(laser_energy=self.laser_energy_slider,
                  laser_radius=self.laser_radius_slider,
                  angular_momentum=self.angular_momentum_slider,
                  pulse_duration=self.pulse_duration_slider)
        def interactive_plot(laser_energy, laser_radius, angular_momentum, pulse_duration):
            with self.output:
                self.output.clear_output(wait=True)
                try:
                    adjusted_pulse_duration = self.calculate_adjusted_pulse_duration(laser_energy, laser_radius, angular_momentum, pulse_duration)
                    print("Adjusted pulse duration:", adjusted_pulse_duration)
                    point_cloud = self.generate_points(laser_energy, laser_radius, angular_momentum, pulse_duration)
                    self.visualize_point_cloud(point_cloud)
                    self.display_metrics(point_cloud)
                except Exception as e:
                    print(f"An error occurred: {e}")

    def calculate_adjusted_pulse_duration(self, laser_energy, laser_radius, angular_momentum, pulse_duration):
        r = np.linalg.norm([laser_energy, laser_radius, angular_momentum])
        schwarzschild_radius = 2 * self.G * r / (self.c ** 2)
        dilation_factor = np.sqrt(1 - (schwarzschild_radius / r))
        return self.wavelength / (self.c * dilation_factor)

    def generate_points(self, laser_energy, laser_radius, angular_momentum, pulse_duration):
        num_points = 100
        points = np.random.uniform(low=-1e9, high=1e9, size=(num_points, 3))
        return points

    def visualize_point_cloud(self, point_cloud):
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2], c='b', marker='.')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title("Ring Laser Time Viewer Point Cloud")
        plt.show()

    def display_metrics(self, point_cloud):
        df = pd.DataFrame(point_cloud, columns=['X', 'Y', 'Z'])

        # Calculate metrics
        metrics = {
            'Mean': df.mean(),
            'Standard Deviation': df.std(),
            'Skewness': skew(df),
            'Kurtosis': kurtosis(df),
            'Correlation Coefficients': df.corr()
        }

        # Display metrics in a tabular format
        for name, metric in metrics.items():
            if isinstance(metric, pd.Series):
                metric = pd.DataFrame(metric).T
            print(f"{name}:")
            display(metric)
            print("\n")

    def display(self):
        display(VBox([self.output]))

# Usage
if __name__ == "__main__":
    viewer = InteractiveRingLaserViewer()
    viewer.display()

interactive(children=(FloatSlider(value=100000.0, description='Laser Energy:', max=1000000.0, step=1.0), Float…